<a href="https://colab.research.google.com/github/frasercrichton/data-investigation-conspiracy-aotearoa/blob/main/analysis/Pattern_of_Life.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download Videos

In [1]:
import logging
import os
import pandas as pd
import cv2
import yt_dlp

from pathlib import Path
from urllib.error import HTTPError
from yt_dlp.utils import ExtractorError, DownloadError
from typing import List, Dict, Optional
from deep_translator import GoogleTranslator

processed_dir = '../data/processed/'
transcription_dir_location = '../data/processed/transcription/'

os.makedirs(processed_dir + 'videos', exist_ok = True)
logger = logging.getLogger(__name__)

os.makedirs(transcription_dir_location, exist_ok = True)

In [17]:
def get_video_duration(filename):

    video = cv2.VideoCapture(filename)

    if video is None or not video.isOpened():
       print('Warning: unable to open video source: ', filename)
       return '###### VIDEO NOT FOUND ###########'

    duration = video.get(cv2.CAP_PROP_POS_MSEC)
    fps = video.get(cv2.CAP_PROP_FPS)      
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count/fps

    minutes = str(int(duration/60))
    seconds = str(round(duration%60))
    

    duration_formatted = f'{minutes}:{seconds:02}' 
    video.release()
    return duration_formatted

videos_translated_df = pd.read_json(processed_dir + 'videos-translated.json', convert_dates=['createTime'])
videos_translated_df['videoId'] = pd.json_normalize(videos_translated_df['video'])['id']
videos_translated_df = videos_translated_df[['createTime',
        'id', 
        'videoId',
        'desc_en', 
        ]]

def add_duration(file_id):
    file_name = processed_dir + 'videos/' + str(file_id) + '.mp4'
    duration = get_video_duration(file_name)
    
    print(duration)
    return duration

videos_translated_df['duration'] = videos_translated_df.apply(lambda row: add_duration(row['videoId']), axis=1)

videos_translated_df.to_json(processed_dir + 'audio-duration.csv')

videos_translated_df['duration']


5:17
2:43
1:27
1:12
9:10
5:16
3:70
6:26
2:56
0:49
1:23
4:52
1:45
2:37
0:32
0:58
2:19
1:80
1:30
2:44
2:30
0:11
2:10
0:54
0:48
1:42
1:17
6:58
1:11
0:39
0:22
6:19
1:47
1:52
1:80
0:47
1:44
0:51
1:33
3:34
0:57
4:11
1:13
5:16
1:26
1:35
1:60
1:11
3:17
1:29
1:70
0:49
0:51
0:45
4:14
3:25
3:28
1:52
2:50
4:33
2:21
0:47
0:20
0:47
4:59
1:57
9:50
0:50
1:30
4:00
6:54
4:12
0:37
0:29
0:39
2:38
1:42
3:39
2:34
0:59
1:52
1:16
7:47
9:51
1:14
1:37
0:23
2:70
0:21
1:20
2:20
1:33
3:12
3:43
0:50
9:48
0:29
0:20
3:10
3:38
4:60
1:41
1:90
1:54
3:60
0:45
0:25
1:18
0:21
1:16
0:50
0:11
1:12
1:90
0:22
2:60
0:28
0:51
0:40
0:35
2:60
1:14
1:57
4:45
3:10
0:11
0:24
3:19
0:53
1:90
1:19
0:28
0:16
0:13
0:46
0:30
0:20
0:17
2:70
2:27
0:24
0:25
0:20
0:38
3:44
0:44
0:24
0:20
0:32
0:13
0:23
2:53
1:44
0:16
0:90
0:11
0:12
0:30
1:38
0:50
0:90
0:15
1:41
2:70
1:28
0:38
1:24
0:37
1:39
1:49
1:50
0:70
3:33
1:50
1:53
1:40
1:40
2:90
2:10
0:23
6:20
3:70
1:16
1:26
0:60
0:40
2:40
1:56
0:15
5:22
2:31
1:41
0:12
0:15
0:17
0:24
3:54
2:80
1:55
0:20


0      5:17
1      2:43
2      1:27
3      1:12
4      9:10
       ... 
219    1:17
220    0:60
221    3:00
222    2:19
223    1:15
Name: duration, Length: 224, dtype: object

## Estimated word count of videos

In [5]:
duration_df = pd.read_json(processed_dir + 'audio-duration.csv', convert_dates=['createTime']) #, convert_dates=['createTime']

def calculate_word_count(duration):
    duration_list = duration.split(':')
    # Fast 160 wpm 
    wpm = 160 / 60
    minute = int(int(duration_list[0]) * 60)
    seconds = int(duration_list[1])
    return round((minute + seconds) * wpm)

duration_df['wordCount'] = duration_df.apply(lambda row :  calculate_word_count(row['duration']), axis=1)  
duration_df

# merge 

,createTime,id,videoId,desc_en,duration,wordCount
0,2023-09-13 18:57:03,7278386520275373000,7278386520275373056,"My interview this evening on ""Cinque Minuti"", ...",5:17,845
1,2023-09-12 16:05:41,7277971263958666000,7277971263958666240,We are giving the Nation a strategy that it ha...,2:43,435
2,2023-09-11 09:02:24,7277491085071388000,7277491085071387648,"Thank you India, congratulations on the succes...",1:27,232
3,2023-09-03 07:28:10,7274498125660704000,7274498125660703744,41 years after the brutal mafia attack that ca...,1:12,192
4,2023-08-12 06:05:11,7266312532665585000,7266312532665584640,Minimum wage: press point after the meeting wi...,9:10,1467
...,...,...,...,...,...,...
219,2022-02-20 17:31:35,7066845965105319000,7066845965105318912,The result of the #greenpass was only to manag...,1:17,205
220,2022-02-18 17:00:34,7066095800291577000,7066095800291576832,We want to defend Italy 🇮🇹,0:60,160
221,2022-02-11 14:53:48,7063465536109235000,7063465536109235200,Let it be clear to everyone: we will never for...,3:00,480
222,2022-02-10 19:07:13,7063159758185827000,7063159758185827328,Citizens and businesses crushed by the crisis ...,2:19,371


Get a subset of videos between the election announcement and results date

In [37]:
start_date = '2022-07-21'
end_date = '2022-09-26'


mask = (duration_df['createTime'] > start_date) & (duration_df['createTime'] <= end_date)
election_dates = duration_df.loc[mask]
print(election_dates.shape)
election_dates.head(75)

election_dates.to_csv(processed_dir + 'topic-map/election-date-posts.csv')

(75, 6)


In [20]:
word_count_df = pd.read_json(processed_dir + '/wordcount.json', convert_dates=['timestamp'])
# word_count_df = pd.DataFrame(zip(timestamps, docs_word_count), columns=['timestamp','word_count'])  
# word_count_df.to_json(processed_dir + '/wordcount.json')

# word_count_df.merge(duration_df, left_on='timestamp', right_on='createTime').head(50)
# 
word_count_df
# duration_df


,timestamp,word_count
0,2023-03-31 18:56:58,204
1,2023-03-31 18:56:58,60
2,2023-03-31 18:56:58,1133
3,2023-03-31 18:56:58,174
4,2023-03-31 18:56:58,1373
...,...,...
161,2023-04-03 18:17:10,34
162,2023-04-03 18:17:10,259
163,2023-04-03 18:17:10,64
164,2023-04-03 18:17:10,353
